# Biblioteki

In [2]:
import numpy as np
import pandas as pd
import json
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import matplotlib.pyplot as plt
plt.style.use("bmh")
plt.rcParams['figure.figsize'] = [15, 10]
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tools.eval_measures import rmse
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV, Lasso
import statsmodels.api as sm

import itertools

# Deklaracje

In [3]:
def plot_ts(ts):
    fig, ax = plt.subplots()
    ts_roll_mean = ts.rolling(window=12).mean()
    ts_roll_std = ts.rolling(window=12).std()
    
    ts.plot(color='blue', label='Original', use_index=False)
    ts_roll_mean.plot(color='red', label="Rolling mean", use_index=False)
    ts_roll_std.plot(color='black', label="Rolling std", use_index=False)
    
    ax.set_xticklabels(ts.index)
    fig.autofmt_xdate()
    
    
    plt.xlabel('Date')
    plt.ylabel('Return ratio')
    plt.show()
    
def perform_dft(ts):
    #Perform Augmented Dickey–Fuller test:
    print('Results of Dickey Fuller Test:')
    dftest = adfuller(y_train, autolag='AIC')
    add_col = []
    values = list(dftest[0:4])
    for key,value in dftest[4].items():
        add_col.append('Critical Value (%s)'%key)
        values.append(value)

    dfoutput = pd.DataFrame(data = [values], index=['Wartość'], columns=['Test Statistic','p-value','#Lags Used','Number of Observations Used']+add_col)
    display(dfoutput)

def decompose(ts):
    decomposition = seasonal_decompose(ts, freq=10) 
    decomposition.plot()
    plt.show()
    
def find_best_hparameters(ts, x, pmax, d, qmax):
    errors_dict = {}
    rmse_value_list = []
    order_list = []
    for p in range(pmax+1):
        for q in range(qmax+1):
            order = (p,d,q)
            try:
                model = SARIMAX(ts, x, order=order, freq='B', trend='c')
                results = model.fit(disp=False)
                rmse_value = rmse(results.fittedvalues, ts)
                rmse_value_list.append(rmse_value)
                order_list.append(order)
            except Exception as e:
                errors_dict[str(order)] = {'error_str': str(e)}
    
    errors = pd.DataFrame.from_dict(errors_dict, orient='index')
    display(errors)
    
    results = sorted([(order, rmse_value) for order, rmse_value in zip(order_list, rmse_value_list)], key = lambda x: x[1])
    best10 = tuple(order for order, rmse_value in results[:10])
    return best10, results

# def find_best_extra_vars(ts, x_array, order, n_max=4):
#     errors_dict = {}
#     rmse_value_list = []
#     order_list = []
#     for i in range(2, n_max+1):
#         print(i)
#         try:
#             model = SARIMAX(ts, order=order, freq='B')
#             results = model.fit()
#             rmse_value = rmse(results.fittedvalues, ts)
#             rmse_value_list.append(rmse_value)
#             order_list.append(order)
#         except Exception as e:
#             errors_dict[str(order)] = {'error_str': str(e)}
    
#     errors = pd.DataFrame.from_dict(errors_dict, orient='index')
#     display(errors)
    
#     results = sorted([(order, rmse_value) for order, rmse_value in zip(order_list, rmse_value_list)], key = lambda x: x[1])
#     best10 = tuple(order for order, rmse_value in results[:10])
#     return best10, results

def select_best_orders_from_best10(ts, x, best10):
    errors_dict = {}
    rmse_value_list = []
    order_list = []
    for order in best10:
        try:
            model = SARIMAX(ts, x, order=order, freq='B', trend='c')
            results = model.fit(disp=False)
            rmse_value = rmse(results.fittedvalues, ts)
            rmse_value_list.append(rmse_value)
            order_list.append(order)
        except Exception as e:
            errors_dict[str(order)] = {'error_str': str(e)}
            
    errors = pd.DataFrame.from_dict(errors_dict, orient='index')
    display(errors)
    
    results = sorted([(order, rmse_value) for order, rmse_value in zip(order_list, rmse_value_list)], key = lambda x: x[1])
    best = results[0][0]
    return best, results

def test_orders(ts, orders):
    errors_dict = {}
    rmse_value_list = []
    order_list = []
    for order in orders:
        try:
            model = ARIMA(ts, order=order, freq='B')
            results = model.fit(disp=False)
            rmse_value = rmse(results.fittedvalues, ts)
            rmse_value_list.append(rmse_value)
            order_list.append(order)
        except Exception as e:
            errors_dict[str(order)] = {'error_str': str(e)}
            
    errors = pd.DataFrame.from_dict(errors_dict, orient='index')
    display(errors)
    
    results = sorted([(order, rmse_value) for order, rmse_value in zip(order_list, rmse_value_list)], key = lambda x: x[1])
    best = results[0][0]
    return best, results

def show_est_plot(ts, x_array=None, order=None, zero=False, naiwny=False, label="Wykres"):
    if zero == True:
        z = [0]*len(ts)
        rmse_value = rmse(ts, z)
        plt.plot(ts.index, z, label="Zero", color='red')
        plt.plot(ts.index, ts, label=label, color='blue')
        plt.title(f'{label} (RMSE: {rmse_value})')
        plt.show()
    elif naiwny == True:
        rmse_value = rmse(ts[1:], ts.shift()[1:])
        plt.plot(ts[1:].index, ts.shift()[1:], label="Naiwny", color='red')
        plt.plot(ts[1:].index, ts[1:], label=label, color='blue')
        plt.title(f'{label} (RMSE: {rmse_value})')
        plt.show()
    else:
        model = SARIMAX(ts, x_array, order=order, freq='B', trend='c')
        results = model.fit(disp=False)
        rmse_value = rmse(results.fittedvalues, ts)
        plt.plot(ts.index, results.fittedvalues, label=label, color='red')
        plt.plot(ts.index, ts, label=label, color='blue')
        plt.title(f'{label} (RMSE: {rmse_value})')
        plt.show()
        

In [4]:
best_results = {}

# Wczytanie zbiorów

In [5]:
train = pd.read_csv('datasets/podzielone/ekonometryczne/wszystkie/train_ekon.csv', index_col=0)
val1 = pd.read_csv('datasets/podzielone/ekonometryczne/wszystkie/valid1_ekon.csv', index_col=0)
val2 = pd.read_csv('datasets/podzielone/ekonometryczne/wszystkie/valid2_ekon.csv', index_col=0)
val3 = pd.read_csv('datasets/podzielone/ekonometryczne/wszystkie/valid3_ekon.csv', index_col=0)
test = pd.read_csv('datasets/podzielone/ekonometryczne/wszystkie/test_ekon.csv', index_col=0)

train.index = pd.to_datetime(train.index)
val1.index = pd.to_datetime(val1.index)
val2.index = pd.to_datetime(val2.index)
val3.index = pd.to_datetime(val3.index)
test.index = pd.to_datetime(test.index)

whole = pd.concat([train, val1, val2, val3, test])

train1 = train.asfreq('B', method='bfill')
train2 = pd.concat([train, val1]).asfreq('B', method='bfill')
train3 = pd.concat([train, val1, val2]).asfreq('B', method='bfill')

val1 = val1.asfreq('B', method='bfill')
val2 = val2.asfreq('B', method='bfill')
val3 = val3.asfreq('B', method='bfill')
test = test.asfreq('B', method='bfill')

display(train.head(3))
display(val1.head(3))
display(val2.head(3))
display(val3.head(3))
display(test.head(3))
display(whole.head(3))

,return ratio,ATR,MOM,CCI,EMA,RSI,Will_R,MFI,DX,PLUS_DM,PPO,AROONOSC,BOP,MINUS_DM,ULTOSC,AD,OBV,DEMA,HT_TRENDLINE,KAMA,MIDPOINT,MIDPRICE,SAR,SAREXT,T3,TEMA,TRIMA,WMA,ADX,ADXR,APO,CMO,MINUS_DI,PLUS_DI,ROC,ROCP,ROCR,ROCR100,ADOSC,NATR,TRANGE,AVGPRICE,MEDPRICE,TYPPRICE,WCLPRICE,HT_DCPERIOD,HT_DCPHASE,HT_TRENDMODE,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,CDL3STARSINSOUTH,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLGAPSIDESIDEWHITE,CDLGRAVESTONEDOJI,CDLHAMMER,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLHIGHWAVE,CDLHIKKAKE,CDLHIKKAKEMOD,CDLHOMINGPIGEON,CDLIDENTICAL3CROWS,CDLINNECK,CDLINVERTEDHAMMER,CDLKICKING,CDLKICKINGBYLENGTH,CDLLADDERBOTTOM,CDLLONGLEGGEDDOJI,CDLLONGLINE,CDLMARUBOZU,CDLMATCHINGLOW,CDLMATHOLD,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLONNECK,CDLPIERCING,CDLRICKSHAWMAN,CDLRISEFALL3METHODS,CDLSEPARATINGLINES,CDLSHOOTINGSTAR,CDLSHORTLINE,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS,BETA,CORREL,LINEARREG,LINEARREG_ANGLE,LINEARREG_INTERCEPT,LINEARREG_SLOPE,STDDEV,TSF,VAR,ADD,DIV,MAX,MAXINDEX,MIN,MININDEX,MULT,SUB,SUM,ATAN,CEIL,COS,COSH,EXP,FLOOR,LN,LOG10,SIN,SINH,SQRT,TAN,TANH,current ratio,long-term debt/capital,debt/equity ratio,gross_margin,operating_margin,ebig margin,pre-tax profit margin,net_profit_margin,asset_turnover,inventory_turnover_ratio,receiveable_turnover,days_sales_in_receivables,roe,return on tangible equity,roa,roi,book value per share,operating cash flow per share,free cash flow per share,nvidia_,GeForce_,gtx_,GPU_,AMD_,Intel_,deep learning_,Artificial Intelligence_,Machine Learning_,neural network_,Data Science_,natural language processing_,fintech_,Azure_,AWS_,Google Cloud_,TensorFlow_,PyTorch_,MXNet_,blockchain_,cryptocurrency_,bitcoin_,ethereum_,Bitcoin miner_,cryptocurrency miner_,gaming_,e-sport_,Battlefield_,Just Cause_,Assassin's Creed_,Hitman_,Far Cry_,Final Fantasy_,Forza Motorsport_,Call of Duty_,Witcher_,Fallout_,gaming PC_,Nvidia Shield_,GTA_,Python_,Twitch_,nvidia_news,gtx_news,GPU_news,AMD_news,deep learning_news,Artificial Intelligence_news,Data Science_news,fintech_news,AWS_news,blockchain_news,bitcoin_news,ethereum_news,gaming_news,e-sport_news,Battlefield_news,Just Cause_news,Assassin's Creed_news,Hitman_news,Far Cry_news,Final Fantasy_news,Forza Motorsport_news,Call of Duty_news,Witcher_news,Fallout_news,gaming PC_news,nvidia__scaled,GeForce__scaled,gtx__scaled,GPU__scaled,AMD__scaled,Intel__scaled,deep learning__scaled,Artificial Intelligence__scaled,Machine Learning__scaled,neural network__scaled,Data Science__scaled,natural language processing__scaled,fintech__scaled,Azure__scaled,AWS__scaled,Google Cloud__scaled,MXNet__scaled,Bitcoin miner__scaled,gaming__scaled,e-sport__scaled,Battlefield__scaled,Just Cause__scaled,Assassin's Creed__scaled,Hitman__scaled,Far Cry__scaled,Final Fantasy__scaled,Forza Motorsport__scaled,Call of Duty__scaled,Witcher__scaled,Fallout__scaled,gaming PC__scaled,GTA__scaled,Python__scaled,Twitch__scaled,nvidia_news_scaled,gtx_news_scaled,GPU_news_scaled,AMD_news_scaled,Artificial Intelligence_news_scaled,AWS_news_scaled,gaming_news_scaled,Battlefield_news_scaled,Just Cause_news_scaled,Assassin's Creed_news_scaled,Hitman_news_scaled,Far Cry_news_scaled,Final Fantasy_news_scaled,Forza Motorsport_news_scaled,Call of Duty_news_scaled,Witcher_news_scaled,Fallout_news_scaled,gaming PC_news_scaled,arxiv,BTC,AMD,SandP,NasdaqTech,ATVI_gaming,TTWO_gaming,UBSFY_gaming,super_gry,fps,"NVIDIA_news_publikacja_(0, 1]","NVIDIA_news_publikacja_(1, 3]","NVIDIA_news_publikacja_(3, 6]","NVIDIA_news_publikacja_(6, 100]","NVIDIA_news_inni_o_nich_pisza_(0, 1]","NVIDIA_news_inni_o_nich_pisza_(1, 3]","NVIDIA_news_inni_o_nich_pisza_(3, 6]","ogloszenie_nowej_karty_(0, 1]"

,return ratio,ATR,MOM,CCI,EMA,RSI,Will_R,MFI,DX,PLUS_DM,PPO,AROONOSC,BOP,MINUS_DM,ULTOSC,AD,OBV,DEMA,HT_TRENDLINE,KAMA,MIDPOINT,MIDPRICE,SAR,SAREXT,T3,TEMA,TRIMA,WMA,ADX,ADXR,APO,CMO,MINUS_DI,PLUS_DI,ROC,ROCP,ROCR,ROCR100,ADOSC,NATR,TRANGE,AVGPRICE,MEDPRICE,TYPPRICE,WCLPRICE,HT_DCPERIOD,HT_DCPHASE,HT_TRENDMODE,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,CDL3STARSINSOUTH,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLGAPSIDESIDEWHITE,CDLGRAVESTONEDOJI,CDLHAMMER,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLHIGHWAVE,CDLHIKKAKE,CDLHIKKAKEMOD,CDLHOMINGPIGEON,CDLIDENTICAL3CROWS,CDLINNECK,CDLINVERTEDHAMMER,CDLKICKING,CDLKICKINGBYLENGTH,CDLLADDERBOTTOM,CDLLONGLEGGEDDOJI,CDLLONGLINE,CDLMARUBOZU,CDLMATCHINGLOW,CDLMATHOLD,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLONNECK,CDLPIERCING,CDLRICKSHAWMAN,CDLRISEFALL3METHODS,CDLSEPARATINGLINES,CDLSHOOTINGSTAR,CDLSHORTLINE,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS,BETA,CORREL,LINEARREG,LINEARREG_ANGLE,LINEARREG_INTERCEPT,LINEARREG_SLOPE,STDDEV,TSF,VAR,ADD,DIV,MAX,MAXINDEX,MIN,MININDEX,MULT,SUB,SUM,ATAN,CEIL,COS,COSH,EXP,FLOOR,LN,LOG10,SIN,SINH,SQRT,TAN,TANH,current ratio,long-term debt/capital,debt/equity ratio,gross_margin,operating_margin,ebig margin,pre-tax profit margin,net_profit_margin,asset_turnover,inventory_turnover_ratio,receiveable_turnover,days_sales_in_receivables,roe,return on tangible equity,roa,roi,book value per share,operating cash flow per share,free cash flow per share,nvidia_,GeForce_,gtx_,GPU_,AMD_,Intel_,deep learning_,Artificial Intelligence_,Machine Learning_,neural network_,Data Science_,natural language processing_,fintech_,Azure_,AWS_,Google Cloud_,TensorFlow_,PyTorch_,MXNet_,blockchain_,cryptocurrency_,bitcoin_,ethereum_,Bitcoin miner_,cryptocurrency miner_,gaming_,e-sport_,Battlefield_,Just Cause_,Assassin's Creed_,Hitman_,Far Cry_,Final Fantasy_,Forza Motorsport_,Call of Duty_,Witcher_,Fallout_,gaming PC_,Nvidia Shield_,GTA_,Python_,Twitch_,nvidia_news,gtx_news,GPU_news,AMD_news,deep learning_news,Artificial Intelligence_news,Data Science_news,fintech_news,AWS_news,blockchain_news,bitcoin_news,ethereum_news,gaming_news,e-sport_news,Battlefield_news,Just Cause_news,Assassin's Creed_news,Hitman_news,Far Cry_news,Final Fantasy_news,Forza Motorsport_news,Call of Duty_news,Witcher_news,Fallout_news,gaming PC_news,nvidia__scaled,GeForce__scaled,gtx__scaled,GPU__scaled,AMD__scaled,Intel__scaled,deep learning__scaled,Artificial Intelligence__scaled,Machine Learning__scaled,neural network__scaled,Data Science__scaled,natural language processing__scaled,fintech__scaled,Azure__scaled,AWS__scaled,Google Cloud__scaled,MXNet__scaled,Bitcoin miner__scaled,gaming__scaled,e-sport__scaled,Battlefield__scaled,Just Cause__scaled,Assassin's Creed__scaled,Hitman__scaled,Far Cry__scaled,Final Fantasy__scaled,Forza Motorsport__scaled,Call of Duty__scaled,Witcher__scaled,Fallout__scaled,gaming PC__scaled,GTA__scaled,Python__scaled,Twitch__scaled,nvidia_news_scaled,gtx_news_scaled,GPU_news_scaled,AMD_news_scaled,Artificial Intelligence_news_scaled,AWS_news_scaled,gaming_news_scaled,Battlefield_news_scaled,Just Cause_news_scaled,Assassin's Creed_news_scaled,Hitman_news_scaled,Far Cry_news_scaled,Final Fantasy_news_scaled,Forza Motorsport_news_scaled,Call of Duty_news_scaled,Witcher_news_scaled,Fallout_news_scaled,gaming PC_news_scaled,arxiv,BTC,AMD,SandP,NasdaqTech,ATVI_gaming,TTWO_gaming,UBSFY_gaming,super_gry,fps,"NVIDIA_news_publikacja_(0, 1]","NVIDIA_news_publikacja_(1, 3]","NVIDIA_news_publikacja_(3, 6]","NVIDIA_news_publikacja_(6, 100]","NVIDIA_news_inni_o_nich_pisza_(0, 1]","NVIDIA_news_inni_o_nich_pisza_(1, 3]","NVIDIA_news_inni_o_nich_pisza_(3, 6]","ogloszenie_nowej_karty_(0, 1]"

,return ratio,ATR,MOM,CCI,EMA,RSI,Will_R,MFI,DX,PLUS_DM,PPO,AROONOSC,BOP,MINUS_DM,ULTOSC,AD,OBV,DEMA,HT_TRENDLINE,KAMA,MIDPOINT,MIDPRICE,SAR,SAREXT,T3,TEMA,TRIMA,WMA,ADX,ADXR,APO,CMO,MINUS_DI,PLUS_DI,ROC,ROCP,ROCR,ROCR100,ADOSC,NATR,TRANGE,AVGPRICE,MEDPRICE,TYPPRICE,WCLPRICE,HT_DCPERIOD,HT_DCPHASE,HT_TRENDMODE,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,CDL3STARSINSOUTH,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLGAPSIDESIDEWHITE,CDLGRAVESTONEDOJI,CDLHAMMER,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLHIGHWAVE,CDLHIKKAKE,CDLHIKKAKEMOD,CDLHOMINGPIGEON,CDLIDENTICAL3CROWS,CDLINNECK,CDLINVERTEDHAMMER,CDLKICKING,CDLKICKINGBYLENGTH,CDLLADDERBOTTOM,CDLLONGLEGGEDDOJI,CDLLONGLINE,CDLMARUBOZU,CDLMATCHINGLOW,CDLMATHOLD,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLONNECK,CDLPIERCING,CDLRICKSHAWMAN,CDLRISEFALL3METHODS,CDLSEPARATINGLINES,CDLSHOOTINGSTAR,CDLSHORTLINE,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS,BETA,CORREL,LINEARREG,LINEARREG_ANGLE,LINEARREG_INTERCEPT,LINEARREG_SLOPE,STDDEV,TSF,VAR,ADD,DIV,MAX,MAXINDEX,MIN,MININDEX,MULT,SUB,SUM,ATAN,CEIL,COS,COSH,EXP,FLOOR,LN,LOG10,SIN,SINH,SQRT,TAN,TANH,current ratio,long-term debt/capital,debt/equity ratio,gross_margin,operating_margin,ebig margin,pre-tax profit margin,net_profit_margin,asset_turnover,inventory_turnover_ratio,receiveable_turnover,days_sales_in_receivables,roe,return on tangible equity,roa,roi,book value per share,operating cash flow per share,free cash flow per share,nvidia_,GeForce_,gtx_,GPU_,AMD_,Intel_,deep learning_,Artificial Intelligence_,Machine Learning_,neural network_,Data Science_,natural language processing_,fintech_,Azure_,AWS_,Google Cloud_,TensorFlow_,PyTorch_,MXNet_,blockchain_,cryptocurrency_,bitcoin_,ethereum_,Bitcoin miner_,cryptocurrency miner_,gaming_,e-sport_,Battlefield_,Just Cause_,Assassin's Creed_,Hitman_,Far Cry_,Final Fantasy_,Forza Motorsport_,Call of Duty_,Witcher_,Fallout_,gaming PC_,Nvidia Shield_,GTA_,Python_,Twitch_,nvidia_news,gtx_news,GPU_news,AMD_news,deep learning_news,Artificial Intelligence_news,Data Science_news,fintech_news,AWS_news,blockchain_news,bitcoin_news,ethereum_news,gaming_news,e-sport_news,Battlefield_news,Just Cause_news,Assassin's Creed_news,Hitman_news,Far Cry_news,Final Fantasy_news,Forza Motorsport_news,Call of Duty_news,Witcher_news,Fallout_news,gaming PC_news,nvidia__scaled,GeForce__scaled,gtx__scaled,GPU__scaled,AMD__scaled,Intel__scaled,deep learning__scaled,Artificial Intelligence__scaled,Machine Learning__scaled,neural network__scaled,Data Science__scaled,natural language processing__scaled,fintech__scaled,Azure__scaled,AWS__scaled,Google Cloud__scaled,MXNet__scaled,Bitcoin miner__scaled,gaming__scaled,e-sport__scaled,Battlefield__scaled,Just Cause__scaled,Assassin's Creed__scaled,Hitman__scaled,Far Cry__scaled,Final Fantasy__scaled,Forza Motorsport__scaled,Call of Duty__scaled,Witcher__scaled,Fallout__scaled,gaming PC__scaled,GTA__scaled,Python__scaled,Twitch__scaled,nvidia_news_scaled,gtx_news_scaled,GPU_news_scaled,AMD_news_scaled,Artificial Intelligence_news_scaled,AWS_news_scaled,gaming_news_scaled,Battlefield_news_scaled,Just Cause_news_scaled,Assassin's Creed_news_scaled,Hitman_news_scaled,Far Cry_news_scaled,Final Fantasy_news_scaled,Forza Motorsport_news_scaled,Call of Duty_news_scaled,Witcher_news_scaled,Fallout_news_scaled,gaming PC_news_scaled,arxiv,BTC,AMD,SandP,NasdaqTech,ATVI_gaming,TTWO_gaming,UBSFY_gaming,super_gry,fps,"NVIDIA_news_publikacja_(0, 1]","NVIDIA_news_publikacja_(1, 3]","NVIDIA_news_publikacja_(3, 6]","NVIDIA_news_publikacja_(6, 100]","NVIDIA_news_inni_o_nich_pisza_(0, 1]","NVIDIA_news_inni_o_nich_pisza_(1, 3]","NVIDIA_news_inni_o_nich_pisza_(3, 6]","ogloszenie_nowej_karty_(0, 1]"

,return ratio,ATR,MOM,CCI,EMA,RSI,Will_R,MFI,DX,PLUS_DM,PPO,AROONOSC,BOP,MINUS_DM,ULTOSC,AD,OBV,DEMA,HT_TRENDLINE,KAMA,MIDPOINT,MIDPRICE,SAR,SAREXT,T3,TEMA,TRIMA,WMA,ADX,ADXR,APO,CMO,MINUS_DI,PLUS_DI,ROC,ROCP,ROCR,ROCR100,ADOSC,NATR,TRANGE,AVGPRICE,MEDPRICE,TYPPRICE,WCLPRICE,HT_DCPERIOD,HT_DCPHASE,HT_TRENDMODE,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,CDL3STARSINSOUTH,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLGAPSIDESIDEWHITE,CDLGRAVESTONEDOJI,CDLHAMMER,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLHIGHWAVE,CDLHIKKAKE,CDLHIKKAKEMOD,CDLHOMINGPIGEON,CDLIDENTICAL3CROWS,CDLINNECK,CDLINVERTEDHAMMER,CDLKICKING,CDLKICKINGBYLENGTH,CDLLADDERBOTTOM,CDLLONGLEGGEDDOJI,CDLLONGLINE,CDLMARUBOZU,CDLMATCHINGLOW,CDLMATHOLD,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLONNECK,CDLPIERCING,CDLRICKSHAWMAN,CDLRISEFALL3METHODS,CDLSEPARATINGLINES,CDLSHOOTINGSTAR,CDLSHORTLINE,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS,BETA,CORREL,LINEARREG,LINEARREG_ANGLE,LINEARREG_INTERCEPT,LINEARREG_SLOPE,STDDEV,TSF,VAR,ADD,DIV,MAX,MAXINDEX,MIN,MININDEX,MULT,SUB,SUM,ATAN,CEIL,COS,COSH,EXP,FLOOR,LN,LOG10,SIN,SINH,SQRT,TAN,TANH,current ratio,long-term debt/capital,debt/equity ratio,gross_margin,operating_margin,ebig margin,pre-tax profit margin,net_profit_margin,asset_turnover,inventory_turnover_ratio,receiveable_turnover,days_sales_in_receivables,roe,return on tangible equity,roa,roi,book value per share,operating cash flow per share,free cash flow per share,nvidia_,GeForce_,gtx_,GPU_,AMD_,Intel_,deep learning_,Artificial Intelligence_,Machine Learning_,neural network_,Data Science_,natural language processing_,fintech_,Azure_,AWS_,Google Cloud_,TensorFlow_,PyTorch_,MXNet_,blockchain_,cryptocurrency_,bitcoin_,ethereum_,Bitcoin miner_,cryptocurrency miner_,gaming_,e-sport_,Battlefield_,Just Cause_,Assassin's Creed_,Hitman_,Far Cry_,Final Fantasy_,Forza Motorsport_,Call of Duty_,Witcher_,Fallout_,gaming PC_,Nvidia Shield_,GTA_,Python_,Twitch_,nvidia_news,gtx_news,GPU_news,AMD_news,deep learning_news,Artificial Intelligence_news,Data Science_news,fintech_news,AWS_news,blockchain_news,bitcoin_news,ethereum_news,gaming_news,e-sport_news,Battlefield_news,Just Cause_news,Assassin's Creed_news,Hitman_news,Far Cry_news,Final Fantasy_news,Forza Motorsport_news,Call of Duty_news,Witcher_news,Fallout_news,gaming PC_news,nvidia__scaled,GeForce__scaled,gtx__scaled,GPU__scaled,AMD__scaled,Intel__scaled,deep learning__scaled,Artificial Intelligence__scaled,Machine Learning__scaled,neural network__scaled,Data Science__scaled,natural language processing__scaled,fintech__scaled,Azure__scaled,AWS__scaled,Google Cloud__scaled,MXNet__scaled,Bitcoin miner__scaled,gaming__scaled,e-sport__scaled,Battlefield__scaled,Just Cause__scaled,Assassin's Creed__scaled,Hitman__scaled,Far Cry__scaled,Final Fantasy__scaled,Forza Motorsport__scaled,Call of Duty__scaled,Witcher__scaled,Fallout__scaled,gaming PC__scaled,GTA__scaled,Python__scaled,Twitch__scaled,nvidia_news_scaled,gtx_news_scaled,GPU_news_scaled,AMD_news_scaled,Artificial Intelligence_news_scaled,AWS_news_scaled,gaming_news_scaled,Battlefield_news_scaled,Just Cause_news_scaled,Assassin's Creed_news_scaled,Hitman_news_scaled,Far Cry_news_scaled,Final Fantasy_news_scaled,Forza Motorsport_news_scaled,Call of Duty_news_scaled,Witcher_news_scaled,Fallout_news_scaled,gaming PC_news_scaled,arxiv,BTC,AMD,SandP,NasdaqTech,ATVI_gaming,TTWO_gaming,UBSFY_gaming,super_gry,fps,"NVIDIA_news_publikacja_(0, 1]","NVIDIA_news_publikacja_(1, 3]","NVIDIA_news_publikacja_(3, 6]","NVIDIA_news_publikacja_(6, 100]","NVIDIA_news_inni_o_nich_pisza_(0, 1]","NVIDIA_news_inni_o_nich_pisza_(1, 3]","NVIDIA_news_inni_o_nich_pisza_(3, 6]","ogloszenie_nowej_karty_(0, 1]"

,return ratio,ATR,MOM,CCI,EMA,RSI,Will_R,MFI,DX,PLUS_DM,PPO,AROONOSC,BOP,MINUS_DM,ULTOSC,AD,OBV,DEMA,HT_TRENDLINE,KAMA,MIDPOINT,MIDPRICE,SAR,SAREXT,T3,TEMA,TRIMA,WMA,ADX,ADXR,APO,CMO,MINUS_DI,PLUS_DI,ROC,ROCP,ROCR,ROCR100,ADOSC,NATR,TRANGE,AVGPRICE,MEDPRICE,TYPPRICE,WCLPRICE,HT_DCPERIOD,HT_DCPHASE,HT_TRENDMODE,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,CDL3STARSINSOUTH,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLGAPSIDESIDEWHITE,CDLGRAVESTONEDOJI,CDLHAMMER,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLHIGHWAVE,CDLHIKKAKE,CDLHIKKAKEMOD,CDLHOMINGPIGEON,CDLIDENTICAL3CROWS,CDLINNECK,CDLINVERTEDHAMMER,CDLKICKING,CDLKICKINGBYLENGTH,CDLLADDERBOTTOM,CDLLONGLEGGEDDOJI,CDLLONGLINE,CDLMARUBOZU,CDLMATCHINGLOW,CDLMATHOLD,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLONNECK,CDLPIERCING,CDLRICKSHAWMAN,CDLRISEFALL3METHODS,CDLSEPARATINGLINES,CDLSHOOTINGSTAR,CDLSHORTLINE,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS,BETA,CORREL,LINEARREG,LINEARREG_ANGLE,LINEARREG_INTERCEPT,LINEARREG_SLOPE,STDDEV,TSF,VAR,ADD,DIV,MAX,MAXINDEX,MIN,MININDEX,MULT,SUB,SUM,ATAN,CEIL,COS,COSH,EXP,FLOOR,LN,LOG10,SIN,SINH,SQRT,TAN,TANH,current ratio,long-term debt/capital,debt/equity ratio,gross_margin,operating_margin,ebig margin,pre-tax profit margin,net_profit_margin,asset_turnover,inventory_turnover_ratio,receiveable_turnover,days_sales_in_receivables,roe,return on tangible equity,roa,roi,book value per share,operating cash flow per share,free cash flow per share,nvidia_,GeForce_,gtx_,GPU_,AMD_,Intel_,deep learning_,Artificial Intelligence_,Machine Learning_,neural network_,Data Science_,natural language processing_,fintech_,Azure_,AWS_,Google Cloud_,TensorFlow_,PyTorch_,MXNet_,blockchain_,cryptocurrency_,bitcoin_,ethereum_,Bitcoin miner_,cryptocurrency miner_,gaming_,e-sport_,Battlefield_,Just Cause_,Assassin's Creed_,Hitman_,Far Cry_,Final Fantasy_,Forza Motorsport_,Call of Duty_,Witcher_,Fallout_,gaming PC_,Nvidia Shield_,GTA_,Python_,Twitch_,nvidia_news,gtx_news,GPU_news,AMD_news,deep learning_news,Artificial Intelligence_news,Data Science_news,fintech_news,AWS_news,blockchain_news,bitcoin_news,ethereum_news,gaming_news,e-sport_news,Battlefield_news,Just Cause_news,Assassin's Creed_news,Hitman_news,Far Cry_news,Final Fantasy_news,Forza Motorsport_news,Call of Duty_news,Witcher_news,Fallout_news,gaming PC_news,nvidia__scaled,GeForce__scaled,gtx__scaled,GPU__scaled,AMD__scaled,Intel__scaled,deep learning__scaled,Artificial Intelligence__scaled,Machine Learning__scaled,neural network__scaled,Data Science__scaled,natural language processing__scaled,fintech__scaled,Azure__scaled,AWS__scaled,Google Cloud__scaled,MXNet__scaled,Bitcoin miner__scaled,gaming__scaled,e-sport__scaled,Battlefield__scaled,Just Cause__scaled,Assassin's Creed__scaled,Hitman__scaled,Far Cry__scaled,Final Fantasy__scaled,Forza Motorsport__scaled,Call of Duty__scaled,Witcher__scaled,Fallout__scaled,gaming PC__scaled,GTA__scaled,Python__scaled,Twitch__scaled,nvidia_news_scaled,gtx_news_scaled,GPU_news_scaled,AMD_news_scaled,Artificial Intelligence_news_scaled,AWS_news_scaled,gaming_news_scaled,Battlefield_news_scaled,Just Cause_news_scaled,Assassin's Creed_news_scaled,Hitman_news_scaled,Far Cry_news_scaled,Final Fantasy_news_scaled,Forza Motorsport_news_scaled,Call of Duty_news_scaled,Witcher_news_scaled,Fallout_news_scaled,gaming PC_news_scaled,arxiv,BTC,AMD,SandP,NasdaqTech,ATVI_gaming,TTWO_gaming,UBSFY_gaming,super_gry,fps,"NVIDIA_news_publikacja_(0, 1]","NVIDIA_news_publikacja_(1, 3]","NVIDIA_news_publikacja_(3, 6]","NVIDIA_news_publikacja_(6, 100]","NVIDIA_news_inni_o_nich_pisza_(0, 1]","NVIDIA_news_inni_o_nich_pisza_(1, 3]","NVIDIA_news_inni_o_nich_pisza_(3, 6]","ogloszenie_nowej_karty_(0, 1]"

,return ratio,ATR,MOM,CCI,EMA,RSI,Will_R,MFI,DX,PLUS_DM,PPO,AROONOSC,BOP,MINUS_DM,ULTOSC,AD,OBV,DEMA,HT_TRENDLINE,KAMA,MIDPOINT,MIDPRICE,SAR,SAREXT,T3,TEMA,TRIMA,WMA,ADX,ADXR,APO,CMO,MINUS_DI,PLUS_DI,ROC,ROCP,ROCR,ROCR100,ADOSC,NATR,TRANGE,AVGPRICE,MEDPRICE,TYPPRICE,WCLPRICE,HT_DCPERIOD,HT_DCPHASE,HT_TRENDMODE,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,CDL3STARSINSOUTH,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLGAPSIDESIDEWHITE,CDLGRAVESTONEDOJI,CDLHAMMER,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLHIGHWAVE,CDLHIKKAKE,CDLHIKKAKEMOD,CDLHOMINGPIGEON,CDLIDENTICAL3CROWS,CDLINNECK,CDLINVERTEDHAMMER,CDLKICKING,CDLKICKINGBYLENGTH,CDLLADDERBOTTOM,CDLLONGLEGGEDDOJI,CDLLONGLINE,CDLMARUBOZU,CDLMATCHINGLOW,CDLMATHOLD,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLONNECK,CDLPIERCING,CDLRICKSHAWMAN,CDLRISEFALL3METHODS,CDLSEPARATINGLINES,CDLSHOOTINGSTAR,CDLSHORTLINE,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS,BETA,CORREL,LINEARREG,LINEARREG_ANGLE,LINEARREG_INTERCEPT,LINEARREG_SLOPE,STDDEV,TSF,VAR,ADD,DIV,MAX,MAXINDEX,MIN,MININDEX,MULT,SUB,SUM,ATAN,CEIL,COS,COSH,EXP,FLOOR,LN,LOG10,SIN,SINH,SQRT,TAN,TANH,current ratio,long-term debt/capital,debt/equity ratio,gross_margin,operating_margin,ebig margin,pre-tax profit margin,net_profit_margin,asset_turnover,inventory_turnover_ratio,receiveable_turnover,days_sales_in_receivables,roe,return on tangible equity,roa,roi,book value per share,operating cash flow per share,free cash flow per share,nvidia_,GeForce_,gtx_,GPU_,AMD_,Intel_,deep learning_,Artificial Intelligence_,Machine Learning_,neural network_,Data Science_,natural language processing_,fintech_,Azure_,AWS_,Google Cloud_,TensorFlow_,PyTorch_,MXNet_,blockchain_,cryptocurrency_,bitcoin_,ethereum_,Bitcoin miner_,cryptocurrency miner_,gaming_,e-sport_,Battlefield_,Just Cause_,Assassin's Creed_,Hitman_,Far Cry_,Final Fantasy_,Forza Motorsport_,Call of Duty_,Witcher_,Fallout_,gaming PC_,Nvidia Shield_,GTA_,Python_,Twitch_,nvidia_news,gtx_news,GPU_news,AMD_news,deep learning_news,Artificial Intelligence_news,Data Science_news,fintech_news,AWS_news,blockchain_news,bitcoin_news,ethereum_news,gaming_news,e-sport_news,Battlefield_news,Just Cause_news,Assassin's Creed_news,Hitman_news,Far Cry_news,Final Fantasy_news,Forza Motorsport_news,Call of Duty_news,Witcher_news,Fallout_news,gaming PC_news,nvidia__scaled,GeForce__scaled,gtx__scaled,GPU__scaled,AMD__scaled,Intel__scaled,deep learning__scaled,Artificial Intelligence__scaled,Machine Learning__scaled,neural network__scaled,Data Science__scaled,natural language processing__scaled,fintech__scaled,Azure__scaled,AWS__scaled,Google Cloud__scaled,MXNet__scaled,Bitcoin miner__scaled,gaming__scaled,e-sport__scaled,Battlefield__scaled,Just Cause__scaled,Assassin's Creed__scaled,Hitman__scaled,Far Cry__scaled,Final Fantasy__scaled,Forza Motorsport__scaled,Call of Duty__scaled,Witcher__scaled,Fallout__scaled,gaming PC__scaled,GTA__scaled,Python__scaled,Twitch__scaled,nvidia_news_scaled,gtx_news_scaled,GPU_news_scaled,AMD_news_scaled,Artificial Intelligence_news_scaled,AWS_news_scaled,gaming_news_scaled,Battlefield_news_scaled,Just Cause_news_scaled,Assassin's Creed_news_scaled,Hitman_news_scaled,Far Cry_news_scaled,Final Fantasy_news_scaled,Forza Motorsport_news_scaled,Call of Duty_news_scaled,Witcher_news_scaled,Fallout_news_scaled,gaming PC_news_scaled,arxiv,BTC,AMD,SandP,NasdaqTech,ATVI_gaming,TTWO_gaming,UBSFY_gaming,super_gry,fps,"NVIDIA_news_publikacja_(0, 1]","NVIDIA_news_publikacja_(1, 3]","NVIDIA_news_publikacja_(3, 6]","NVIDIA_news_publikacja_(6, 100]","NVIDIA_news_inni_o_nich_pisza_(0, 1]","NVIDIA_news_inni_o_nich_pisza_(1, 3]","NVIDIA_news_inni_o_nich_pisza_(3, 6]","ogloszenie_nowej_karty_(0, 1]"

# Wybór X

In [6]:
first_iter_columns = [
    'gross_margin',
    'operating_margin',
    'ebig margin',
    'pre-tax profit margin',
    'receiveable_turnover',
    'days_sales_in_receivables',
    'roe',
    'return on tangible equity',
    'roi',
    'book value per share',
    'EMA',
    'Will_R',
    'DEMA',
    'HT_TRENDLINE',
    'KAMA',
    'MIDPOINT',
    'MIDPRICE',
    'T3',
    'TEMA',
    'TRIMA',
    'WMA',
    'PLUS_DI',
    'NATR',
    'AVGPRICE',
    'MEDPRICE',
    'TYPPRICE',
    'WCLPRICE',
    'LINEARREG',
    'LINEARREG_INTERCEPT',
    'TSF',
    'ADD',
    'MAX',
    'MIN',
    'MININDEX',
    'MULT',
    'SUM',
    'ATAN',
    'CEIL',
    'FLOOR',
    'TAN',
    'AMD_',
    'Intel_',
    'Data Science_',
    'fintech_',
    'cryptocurrency_',
    'bitcoin_',
    'cryptocurrency miner_',
    'Battlefield_',
    'Just Cause_',
    "Assassin's Creed_",
    'gaming PC_',
    'blockchain_news',
    'gaming_news',
    'Battlefield_news',
    "Assassin's Creed_news",
    'AMD__scaled',
    'Intel__scaled',
    'Data Science__scaled',
    'fintech__scaled',
    'Azure__scaled',
    'e-sport__scaled',
    'Battlefield__scaled',
    'Just Cause__scaled',
    "Assassin's Creed__scaled",
    'Forza Motorsport__scaled',
    'gaming PC__scaled',
    'gaming_news_scaled',
    'Battlefield_news_scaled',
    'Just Cause_news_scaled',
    "Assassin's Creed_news_scaled",
    'NasdaqTech',
    'TTWO_gaming',
    'UBSFY_gaming',
    'fps',
    'NVIDIA_news_publikacja_(3, 6]',
    'UBSFY_gaming_rr',
    'AMD_rr',
    'BTC_rolling_variance',
    'ATVI_gaming_rolling_variance',
    'UBSFY_gaming_rolling_variance',
    'ATVI_gaming_diff_1',
    'TTWO_gaming_diff_1',
    'UBSFY_gaming_diff_1',
    'fps_transformed_interpolated_diff_1_shift_2',
    'fps_interpolated_transformed_diff_1_shift_2',
    'rolling variance',
    'close',
    'day_of_week',
    'day_of_year',
    'week'
]

## Trening 1

In [17]:
train1["day_of_week"] = train1.index.dayofweek
train1["day_of_year"] = train1.index.dayofweek
train1["week"] = train1.index.week
train1["quarter"] = train1.index.quarter

val1["day_of_week"] = val1.index.dayofweek
val1["day_of_year"] = val1.index.dayofweek
val1["week"] = val1.index.week
val1["quarter"] = val1.index.quarter

y_train = train1['return ratio']
x_train = train1.drop('return ratio', axis=1)

y_test = val1['return ratio']
x_test = val1.drop('return ratio', axis=1)

In [22]:
scores = []
variables = []
for alpha in [0.005*x for x in range(1, 2*10**2)]:
    print(alpha)
    try:
        clf = Lasso(alpha=alpha)
        featureSelection = SelectFromModel(clf)
        featureSelection.fit(x_train, y_train)
        train1_variables = x_train.columns[featureSelection.get_support()].tolist()
        if len(train1_variables) >= 1 and train1_variables not in variables:
            scores.append((rmse(clf.fit(x_train, y_train).predict(x_test), y_test), alpha, train1_variables))
            variables.append(train1_variables)
    except Exception as e: raise e

# train1_variables = sorted(scores, key=lambda x: x[0])[0][2]
sorted(scores, key=lambda x: x[0])[:5]

0.005


/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.01


/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.015


...

0.99


/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.995


/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[(109633500773.34995,
  0.005,
  ['Will_R',
   'AROONOSC',
   'ADX',
   'CMO',
   'CDL3INSIDE',
   'CDLENGULFING',
   'CDLEVENINGDOJISTAR',
   'CDLEVENINGSTAR',
   'CDLHIKKAKE',
   'CDLINVERTEDHAMMER',
   'CDLMARUBOZU',
   'CDLMATCHINGLOW',
   'CDLPIERCING',
   'CDLSHORTLINE',
   'LINEARREG_ANGLE',
   'MININDEX',
   'MXNet_',
   'e-sport_',
   'Just Cause_',
   'Final Fantasy_',
   'gtx_news',
   'deep learning_news',
   'Artificial Intelligence_news',
   'e-sport_news',
   'Battlefield_news',
   'Hitman_news',
   'gaming PC_news',
   'SandP',
   'fps',
   'NasdaqTech_diff_1',
   'week']),
 (154154645414.892, 0.035, ['MININDEX']),
 (160124464588.17487, 0.03, ['AROONOSC', 'MININDEX']),
 (165732506347.38864,
  0.025,
  ['AROONOSC', 'CDLENGULFING', 'MININDEX', 'SandP']),
 (169017635389.0399,
  0.02,
  ['AROONOSC', 'CDLENGULFING', 'CDLSHORTLINE', 'MININDEX', 'SandP'])]

In [23]:
pmax = 3
qmax = 3
d = 0

for i, (score, alpha, varss) in enumerate(sorted(scores, key=lambda x: x[0])[:5]):
    x_train = train1[varss]
    x_test = val1[varss]
    best10, results = find_best_hparameters(y_train, x_train, pmax=pmax, qmax=qmax, d=d)
#     print(best10)
#     print(results)
#     print('###############################################')
    best, results = select_best_orders_from_best10(y_test, x_test, best10)
    best_results[f'train1_{i}'] = (results, varss)
#     print(best)
#     print(results)

/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/statespace/representation.py:375: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return matrix[[slice(None)]*(matrix.ndim-1) + [0]]
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to 

,error_str
"(1, 0, 1)",Non-stationary starting autoregressive paramet...
"(2, 0, 1)",Non-stationary starting autoregressive paramet...
"(2, 0, 2)",Non-stationary starting autoregressive paramet...
"(3, 0, 2)",non-invertible starting MA parameters found wi...


/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed

,error_str
"(3, 0, 3)",non-invertible starting MA parameters found wi...


/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed

...

""


/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed

,error_str
"(1, 0, 1)",Non-stationary starting autoregressive paramet...
"(2, 0, 1)",Non-stationary starting autoregressive paramet...


/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed

""


## Trening 2

In [10]:
train2["day_of_week"] = train2.index.dayofweek
train2["day_of_year"] = train2.index.dayofweek
train2["week"] = train2.index.week
train2["quarter"] = train2.index.quarter

val2["day_of_week"] = val2.index.dayofweek
val2["day_of_year"] = val2.index.dayofweek
val2["week"] = val2.index.week
val2["quarter"] = val2.index.quarter



y_train = train2['return ratio']
x_train = train2.drop('return ratio', axis=1)

y_test = val2['return ratio']
x_test = val2.drop('return ratio', axis=1)

In [11]:
scores = []
variables = []
for alpha in [0.005*x for x in range(1, 2*10**2)]:
    try:
        clf = Lasso(alpha=alpha)
        featureSelection = SelectFromModel(clf)
        featureSelection.fit(x_train, y_train)
        train2_variables = x_train.columns[featureSelection.get_support()].tolist()
        if len(train2_variables) >= 2 and train2_variables not in variables:
            scores.append((rmse(clf.fit(x_train, y_train).predict(x_test), y_test), alpha, train2_variables))
            variables.append(train2_variables)
    except Exception as e: raise e

# train2_variables = sorted(scores, key=lambda x: x[0])[0][2]
sorted(scores, key=lambda x: x[0])[:5]

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

...

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

[(0.0349757278988923, 0.033, ['SAREXT', 'MININDEX']),
 (0.035183155390526626, 0.0275, ['AROONOSC', 'SAREXT', 'MININDEX']),
 (0.03521477300330499,
  0.025500000000000002,
  ['AROONOSC', 'SAREXT', 'CDLSHORTLINE', 'MININDEX']),
 (0.03538019122238077,
  0.02,
  ['AROONOSC', 'SAREXT', 'CDLENGULFING', 'CDLSHORTLINE', 'MININDEX']),
 (0.035544209510622135,
  0.017,
  ['AROONOSC',
   'SAREXT',
   'CDLENGULFING',
   'CDLSHORTLINE',
   'MININDEX',
   'NasdaqTech_diff_1'])]

In [12]:
pmax = 3
qmax = 3
d = 0

for i, (score, alpha, varss) in enumerate(sorted(scores, key=lambda x: x[0])[:5]):
    x_train = train2[varss]
    x_test = val2[varss]
    best10, results = find_best_hparameters(y_train, x_train, pmax=pmax, qmax=qmax, d=d)
#     print(best10)
#     print(results)
    print('###############################################')
    best, results = select_best_orders_from_best10(y_test, x_test, best10)
    best_results[f'train2_{i}'] = (results, varss)
#     print(best)
#     print(results)

/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/statespace/representation.py:375: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return matrix[[slice(None)]*(matrix.ndim-1) + [0]]
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to 

,error_str
"(1, 0, 1)",Non-stationary starting autoregressive paramet...
"(2, 0, 1)",Non-stationary starting autoregressive paramet...
"(2, 0, 2)",Non-stationary starting autoregressive paramet...


/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


###############################################


/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed

...

/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed

,error_str
"(1, 0, 1)",Non-stationary starting autoregressive paramet...
"(2, 0, 1)",Non-stationary starting autoregressive paramet...


###############################################


/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed

""


## Trening 3

In [24]:
train3["day_of_week"] = train3.index.dayofweek
train3["day_of_year"] = train3.index.dayofweek
train3["week"] = train3.index.week
train3["quarter"] = train3.index.quarter

val3["day_of_week"] = val3.index.dayofweek
val3["day_of_year"] = val3.index.dayofweek
val3["week"] = val3.index.week
val3["quarter"] = val3.index.quarter



y_train = train3['return ratio']
x_train = train3.drop('return ratio', axis=1)

y_test = val3['return ratio']
x_test = val3.drop('return ratio', axis=1)

In [25]:
scores = []
variables = []
for alpha in [0.005*x for x in range(1, 2*10**2)]:
    try:
        clf = Lasso(alpha=alpha)
        featureSelection = SelectFromModel(clf)
        featureSelection.fit(x_train, y_train)
        train3_variables = x_train.columns[featureSelection.get_support()].tolist()
        if len(train3_variables) >= 2 and train3_variables not in variables:
            scores.append((rmse(clf.fit(x_train, y_train).predict(x_test), y_test), alpha, train3_variables))
            variables.append(train3_variables)
    except: pass

sorted(scores, key=lambda x: x[0])[:5]

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not co

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[(56319.51897623796, 0.025, ['AROONOSC', 'MININDEX']),
 (56786.275352212084,
  0.005,
  ['Will_R',
   'AROONOSC',
   'SAREXT',
   'ADX',
   'CMO',
   'CDL3INSIDE',
   'CDLENGULFING',
   'CDLEVENINGDOJISTAR',
   'CDLEVENINGSTAR',
   'CDLGRAVESTONEDOJI',
   'CDLHIKKAKE',
   'CDLHOMINGPIGEON',
   'CDLINVERTEDHAMMER',
   'CDLMARUBOZU',
   'CDLMATCHINGLOW',
   'CDLSHORTLINE',
   'CDLXSIDEGAP3METHODS',
   'LINEARREG_ANGLE',
   'VAR',
   'MININDEX',
   'MXNet_',
   'bitcoin_',
   'e-sport_',
   'Final Fantasy_',
   'deep learning_news',
   'Artificial Intelligence_news',
   'ethereum_news',
   'Battlefield_news',
   'gaming PC_news',
   'SandP',
   'fps',
   'NasdaqTech_diff_1']),
 (57075.017341980405, 0.02, ['AROONOSC', 'CDLSHORTLINE', 'MININDEX']),
 (57568.126241062826,
  0.015,
  ['AROONOSC',
   'SAREXT',
   'CDLENGULFING',
   'CDLHIKKAKE',
   'CDLSHORTLINE',
   'MININDEX',
   'SandP',
   'NasdaqTech_diff_1']),
 (57668.202730048346,
  0.01,
  ['AROONOSC',
   'SAREXT',
   'CDL3INSIDE',
   '

In [ ]:
pmax = 3
qmax = 3
d = 0

for i, (score, alpha, varss) in enumerate(sorted(scores, key=lambda x: x[0])[:5]):
    x_train = train3[varss]
    x_test = val3[varss]
    best10, results = find_best_hparameters(y_train, x_train, pmax=pmax, qmax=qmax, d=d)
#     print(best10)
#     print(results)
#     print('###############################################')
    best, results = select_best_orders_from_best10(y_test, x_test, best10)
    best_results[f'train3_{i}'] = (results, varss)
#     print(best)
#     print(results)

/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/statespace/representation.py:375: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return matrix[[slice(None)]*(matrix.ndim-1) + [0]]
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to 

,error_str
"(1, 0, 1)",Non-stationary starting autoregressive paramet...
"(2, 0, 1)",Non-stationary starting autoregressive paramet...


/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed

""


/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed

## Wyniki selekcji

In [ ]:
best_results

In [ ]:
rmse_values_dict = {}
chosen_one_list = []
for k, (results, variables) in best_results.items():
    for order, rmse_value in results:
        rmse_values = []
        try:
            model = SARIMAX(val1['return ratio'], val1[variables], order=order, freq='B', trend='c')
            results = model.fit(disp=-1)
            rmse_value = rmse(results.fittedvalues, val1['return ratio'])
            rmse_values.append(rmse_value)
        except Exception as e: pass
            
        try:
            model = SARIMAX(val2['return ratio'], val2[variables], order=order, freq='B', trend='c')
            results = model.fit(disp=-1)
            rmse_value = rmse(results.fittedvalues, val2['return ratio'])
            rmse_values.append(rmse_value)
        except: pass
            
        try:
            model = SARIMAX(val3['return ratio'], val3[variables], order=order, freq='B', trend='c')
            results = model.fit(disp=-1)
            rmse_value = rmse(results.fittedvalues, val3['return ratio'])
            rmse_values.append(rmse_value)
        except: pass
        print(rmse_values)
        if len(rmse_values) == 3: chosen_one_list.append((order, variables, np.mean(rmse_values)))

        
chosen_one_list.sort(key=lambda x: x[2])

In [ ]:
chosen_one_list

# Estymacja na walidacji i teście

## Walidacja

In [ ]:
order = (3, 0, 1)
variables = ['days_sales_in_receivables',
   'Will_R',
   'PLUS_DI',
   'TSF',
   'ADD',
   'MAX',
   'MIN',
   'cryptocurrency_',
   'bitcoin_',
   'Just Cause_',
   "Assassin's Creed_",
   'gaming PC_',
   'blockchain_news',
   'gaming_news',
   "Assassin's Creed_news",
   'NasdaqTech',
   'TTWO_gaming',
   'fps',
   'TTWO_gaming_diff_1',
   'close']
y_train_base = train1['return ratio']
x_train_base = train1[variables]
y_test_base = pd.concat([val1, val2, val3]).asfreq('B', method='bfill')['return ratio']
x_test_base = pd.concat([val1, val2, val3]).asfreq('B', method='bfill')[variables]
period = pd.concat([y_train, y_test])

In [ ]:
model = SARIMAX(y_train_base, x_train_base, order=order, freq='B')
results = model.fit()
print(results.summary())

In [ ]:
forecasts = []
for i in range(len(y_test_base)):
    y_train = pd.concat([y_train_base, y_test_base[:i]]).asfreq('B', method='bfill')
    x_train = pd.concat([x_train_base, x_test_base[:i]]).asfreq('B', method='bfill')
    model = SARIMAX(y_train, x_train, order=order, freq='B', trend='c', enforce_invertibility=False, enforce_stationarity=False)
    results = model.fit(disp=False)
    forecasts.append(results.forecast(exog=x_test_base[i:i+1])[0])
forecasts = pd.Series(forecasts, index=y_test_base.index)

In [ ]:
forecasts.plot(color='red', label='Forecast')
y_test_base.plot(color='blue', label='Real')
plt.title(rmse(y_test_base, forecasts))
plt.show()
show_est_plot(y_test_base, naiwny=True, label='Naiwny')
show_est_plot(y_test_base, zero=True, label='Zero')

## Test

In [ ]:
order = (3, 0, 1)
variables = ['days_sales_in_receivables',
   'Will_R',
   'PLUS_DI',
   'TSF',
   'ADD',
   'MAX',
   'MIN',
   'cryptocurrency_',
   'bitcoin_',
   'Just Cause_',
   "Assassin's Creed_",
   'gaming PC_',
   'blockchain_news',
   'gaming_news',
   "Assassin's Creed_news",
   'NasdaqTech',
   'TTWO_gaming',
   'fps',
   'TTWO_gaming_diff_1',
   'close']
y_train_base = pd.concat([train1, val1, val2, val3]).asfreq('B', method='bfill')['return ratio']
x_train_base = pd.concat([train1, val1, val2, val3]).asfreq('B', method='bfill')[variables]
y_test_base = test['return ratio']
x_test_base = test[variables]
period = pd.concat([y_train, y_test])

In [ ]:
model = SARIMAX(y_train_base, x_train_base, order=order, freq='B')
results = model.fit()
print(results.summary())

In [ ]:
forecasts = []
for i in range(len(y_test_base)):
    y_train = pd.concat([y_train_base, y_test_base[:i]]).asfreq('B', method='bfill')
    x_train = pd.concat([x_train_base, x_test_base[:i]]).asfreq('B', method='bfill')
    model = SARIMAX(y_train, x_train, order=order, freq='B', trend='c')
    results = model.fit(disp=False, enforce_invertibility=False)
    forecasts.append(results.forecast(exog=x_test_base[i:i+1])[0])
forecasts = pd.Series(forecasts, index=y_test_base.index)

In [ ]:
forecasts.plot(color='red', label='Forecast')
y_test_base.plot(color='blue', label='Real')
plt.title(rmse(y_test_base, forecasts))
plt.show()
show_est_plot(y_test_base, naiwny=True, label='Naiwny')
show_est_plot(y_test_base, zero=True, label='Zero')